In [ ]:
import pandas as pd
import glob
import os

In [ ]:
# Definir o caminho da pasta onde os arquivos CSV estão localizados
caminho = os.path.join("files", "csv_2", "microinversores")
print(caminho)

In [ ]:
# Função para obter todos os arquivos CSV na pasta
def obter_arquivos_csv(caminho):
    """Retorna uma lista de caminhos dos arquivos CSV no diretório especificado."""
    arquivos = glob.glob(os.path.join(caminho, '*.csv'))
    if not arquivos:
        print("Nenhum arquivo CSV encontrado.")
    return arquivos

In [ ]:
# Função para ler e processar os arquivos CSV
def ler_arquivos_csv(arquivos_csv):
    """Lê todos os arquivos CSV e retorna uma lista de DataFrames."""
    lista = []
    for arquivo in arquivos_csv:
        try:
            df = pd.read_csv(arquivo, index_col=None, header=0, low_memory=False, parse_dates=['Date'])
            lista.append(df)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")
    return lista

In [ ]:
# Função para concatenar os DataFrames
def concatenar_dataframes(lista):
    """Concatena todos os DataFrames em um único DataFrame."""
    if not lista:
        print('Nenhum dado para concatenar.')
        return None
    return pd.concat(lista, ignore_index=True)

In [ ]:
# Função para criar uma nova coluna contendo o nome do microinversor baseado no SN
def adicionar_nome_microinversor(df):
    """
    Adiciona uma coluna 'Microinversor' baseada na coluna 'SN'.
    Associa cada SN a um nome de microinversor único.
    """
    if 'SN' not in df.columns and 'Sn' in df.columns:
        df.rename(columns={'Sn': 'SN'}, inplace=True)

    if 'SN' not in df.columns:
        print("Erro: A coluna 'SN' não foi encontrada no arquivo.")
        return df

    # Criar um dicionário associando os SNs a nomes de microinversores
    microinversores_dict = {sn: f"Microinversor_{idx+1}" for idx, sn in enumerate(df['SN'].unique())}

    # Mapear a coluna SN para os nomes dos microinversores
    df['Microinversor'] = df['SN'].map(microinversores_dict)

    return df

In [ ]:
# Função para salvar o DataFrame atualizado
def salvar_dataframe(df_final, caminho_novo_arquivo):
    """Salva o DataFrame atualizado em um arquivo CSV."""
    try:
        df_final.to_csv(caminho_novo_arquivo, index=False)
        print(f"Arquivo salvo com sucesso: {caminho_novo_arquivo}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")

# Função principal
def main():
    caminho_novo_arquivo = os.path.join(caminho, 'arquivo.csv')

    arquivos_csv = obter_arquivos_csv(caminho)
    if not arquivos_csv:
        return  # Encerra o script se não houver arquivos

    lista = ler_arquivos_csv(arquivos_csv)
    df_novo = concatenar_dataframes(lista)

    if df_novo is not None:
        df_novo = adicionar_nome_microinversor(df_novo)
        display(df_novo)
        salvar_dataframe(df_novo, caminho_novo_arquivo)

In [ ]:
if __name__ == "__main__":
    main()